In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [65]:
epsilon = 1e-8
x = torch.randint(0,10,(5,), dtype=torch.float32)
y = torch.randint(0,10,(5,), dtype=torch.float32)
print(x.shape)
print(y.shape)
x = x.unsqueeze(0)
y = y.unsqueeze(1)
print(x.shape)
print(y.shape)
print(x)
print(y)
diff_matrix = torch.abs(x - y)
diff_matrix
# sim_matrix = 1.0 / diff_matrix

# sim_matrix = F.softmax(sim_matrix, dim=-1) / torch.sqrt(torch.tensor(y.shape[1]))

torch.Size([5])
torch.Size([5])
torch.Size([1, 5])
torch.Size([5, 1])
tensor([[2., 7., 0., 1., 8.]])
tensor([[2.],
        [7.],
        [9.],
        [1.],
        [9.]])


tensor([[0., 5., 2., 1., 6.],
        [5., 0., 7., 6., 1.],
        [7., 2., 9., 8., 1.],
        [1., 6., 1., 0., 7.],
        [7., 2., 9., 8., 1.]])

In [44]:
diff_matrix += epsilon
sim_matrix = 1.0 / diff_matrix
sim_matrix

tensor([[[1.0000e+00, 2.0000e-01, 1.0000e+00, 5.0000e-01, 5.0000e-01],
         [1.6667e-01, 5.0000e-01, 1.6667e-01, 2.0000e-01, 2.0000e-01],
         [1.0000e+00, 2.0000e-01, 1.0000e+00, 5.0000e-01, 5.0000e-01],
         [1.0000e+08, 2.5000e-01, 1.0000e+08, 1.0000e+00, 1.0000e+00],
         [1.0000e+00, 2.0000e-01, 1.0000e+00, 5.0000e-01, 5.0000e-01]]])

In [57]:
sim_matrix = F.softmax(sim_matrix, dim=-1)# / torch.sqrt(torch.tensor(y.shape[1]))
type(sim_matrix.size(-1))

int

In [174]:
num_heads = 2
dim = 3
bsz = 5
alphaq = torch.randint(10,(1,num_heads))
betaq = torch.randint(10,(1,num_heads))
alphak = torch.randint(10,(1,num_heads))
betak = torch.randint(10,(1,num_heads))
x = torch.randint(10,(bsz,dim,1))
print(x.shape)
print(alphaq.shape)
q = (x@alphaq + torch.ones((dim,1),dtype=torch.int64)@betaq).unsqueeze(1)
k = (x@alphak + torch.ones((dim,1), dtype=torch.int64)@betak).unsqueeze(2)
q[0], q[0].size(), k[0], k[0].size(), (q-k)[0], ((q-k)[0]).size()

torch.Size([5, 3, 1])
torch.Size([1, 2])


(tensor([[[33,  9],
          [ 9,  9],
          [33,  9]]]),
 torch.Size([1, 3, 2]),
 tensor([[[65, 65]],
 
         [[23, 17]],
 
         [[65, 65]]]),
 torch.Size([3, 1, 2]),
 tensor([[[-32, -56],
          [-56, -56],
          [-32, -56]],
 
         [[ 10,  -8],
          [-14,  -8],
          [ 10,  -8]],
 
         [[-32, -56],
          [-56, -56],
          [-32, -56]]]),
 torch.Size([3, 3, 2]))

In [172]:
(q-k)[0][:,:,0]

tensor([[ 35, -10,  -1],
        [ 65,  20,  29],
        [ 59,  14,  23]])

In [115]:
h=2
a_q = nn.Parameter(torch.Tensor(1,h))
x = torch.rand((3,1))
(x@a_q).size()

torch.Size([3, 2])

In [20]:
similarity = torch.randint(100,(10,3,3,2), dtype=torch.float32)
print(similarity[0][:,:,0])
similarity=F.softmax(similarity, dim=-2)
print(similarity[0][:,:,0])

tensor([[ 4., 24., 47.],
        [99.,  6., 57.],
        [31., 58., 35.]])
tensor([[2.1151e-19, 1.0262e-10, 1.0000e+00],
        [1.0000e+00, 4.0796e-41, 5.7495e-19],
        [1.8795e-12, 1.0000e+00, 1.0262e-10]])


In [15]:
print(v[0][:,0])

tensor([6, 1, 0])
